In [1]:
import MDAnalysis as mda
import matplotlib.pyplot as plt
import numpy as np
import os

d:\all_programming\MSD\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cwd = os.getcwd()
cwd = cwd.replace('msd_water_box',r"data_files\unwrapped")

graphite_water_psf = cwd + r'\graphite_water.psf'
graphite_trajectory_dcd = cwd + r'\output_per_20ps.dcd'

global_uni = mda.Universe(graphite_water_psf,graphite_trajectory_dcd)
Oxg_uni = global_uni.select_atoms('resname SPCE and name OW')
# Importing data

d:\all_programming\MSD\.venv\Lib\site-packages\MDAnalysis\coordinates\DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [3]:
def calculate_msd(position_dict):
    x_vals = np.array(position_dict['x_vals'])
    y_vals = np.array(position_dict['y_vals'])
    z_vals = np.array(position_dict['z_vals'])
    num_points = len(x_vals)
    msd = np.zeros(num_points)
    for dt in range(1, num_points):
        displacements = np.square(x_vals[dt:] - x_vals[:-dt]) + np.square(y_vals[dt:] - y_vals[:-dt]) + np.square(z_vals[dt:] - z_vals[:-dt])
        msd[dt] = np.mean(displacements)
    return msd

In [4]:
def matrix_addition(array1, array2):
    if len(array1) > len(array2):
        array2 = np.pad(array2, (0, len(array1) - len(array2)), 'constant')
    else:
        array1 = np.pad(array1, (0, len(array2) - len(array1)), 'constant')
    return array1 + array2

In [5]:
partition_height = 65
partition_cuts = 4
partition = []

for i in range(partition_cuts):
    partition.append(65 * ( (i + 1) / partition_cuts))

In [6]:
position_dict = {}
msd_dict = {}

for i in range(partition_cuts):
    msd_dict[i] = {}

atom_num = 0



position_dict[atom_num] = {'location':[],'x_vals':[],'y_vals':[],'z_vals':[]}

for frame in range(len(global_uni.trajectory)):
    global_uni.trajectory[frame]
    oxg_pos = Oxg_uni.positions
    wanted_atom_pos = oxg_pos[atom_num]
    current_z = wanted_atom_pos[2]

    for i in range(len(partition)):
        if current_z < partition[i]:
            position_dict[atom_num]['location'].append(i)
            break

    position_dict[atom_num]['x_vals'].append(wanted_atom_pos[0])
    position_dict[atom_num]['y_vals'].append(wanted_atom_pos[1])
    position_dict[atom_num]['z_vals'].append(wanted_atom_pos[2])



In [17]:
position_dict[atom_num]['new_location'] = []

for i in range(len(position_dict[atom_num]['location'])):
    try:
        current_1 = position_dict[atom_num]['location'][i]

        if i - 1 < 0 or i + 1 == len(position_dict[atom_num]['location']):
            raise KeyError

        past_1 = position_dict[atom_num]['location'][i - 1]
        future_1 = position_dict[atom_num]['location'][i + 1]

        if current_1 != past_1 and past_1 == future_1:
            current_1 = past_1

    except KeyError:
        pass

    position_dict[atom_num]['new_location'].append(current_1)

In [18]:
msd_input_dict = {}

for i in range(len(partition)):
    msd_input_dict[i] = []


In [35]:
for i in range(len(position_dict[atom_num]['location'])):
    if 0 < i - 1:
        if position_dict[atom_num]['location'][i] == position_dict[atom_num]['location'][i - 1]:
            msd_input_dict[position_dict[atom_num]['location'][i]][-1]['x_vals'].append(position_dict[atom_num]['x_vals'][i])
            msd_input_dict[position_dict[atom_num]['location'][i]][-1]['y_vals'].append(position_dict[atom_num]['y_vals'][i])
            msd_input_dict[position_dict[atom_num]['location'][i]][-1]['z_vals'].append(position_dict[atom_num]['z_vals'][i])
        else:
            msd_input_dict[position_dict[atom_num]['location'][i]].append({'x_vals':[],'y_vals':[],'z_vals':[]})
    else:
        msd_input_dict[position_dict[atom_num]['location'][i]].append({'x_vals':[],'y_vals':[],'z_vals':[]})

for key in msd_input_dict.keys():
    empty_indices = []

    for i in range(len(msd_input_dict[key])):
        if len(msd_input_dict[key][i]['x_vals']) == 0:
            empty_indices.append(i)

    for item in reversed(empty_indices):
        del msd_input_dict[key][item]

In [37]:
msd_final_input_dict = {}

for key in msd_input_dict.keys():
    msd_final_input_dict[key] = []

    for item in msd_input_dict[key]:
        msd_final_input_dict[key].append(calculate_msd(item))

In [39]:
max_len = []

for key in msd_final_input_dict.keys():
    temp_len = []

    for item in msd_final_input_dict[key]:
        temp_len.append(len(item))
        
    max_len.append(max(temp_len))
    

In [ ]:
summed_dict = {}

for key in msd_final_input_dict.keys():
    summed_dict[key] = np.zeros(max_len[key])

    for item in msd_final_input_dict[key]:
        summed_dict[key] = matrix_addition(summed_dict[key], item)



In [ ]:
print(summed_dict)

In [40]:
print(max_len)

[44, 44, 44, 32]


In [38]:
print(msd_final_input_dict)

{0: [array([  0.        ,  19.61707687,  37.33045197,  54.28623962,
        70.38471222,  92.03923798, 118.42266846, 135.31750488,
       161.52561951, 181.4460907 , 199.445755  , 212.92657471,
       219.12294006, 218.29504395, 216.53678894, 228.13893127,
       234.38314819, 222.82383728, 225.20942688, 220.32015991,
       209.21688843, 178.36126709, 179.29055786, 178.92837524,
       174.24858093, 154.78353882, 142.33297729, 144.86030579,
       121.45942688, 222.30871582]), array([  0.        ,  18.13752365,  42.03481293,  85.40600586,
       115.90609741, 140.16749573]), array([0.]), array([ 0.        , 34.51122284, 31.83320045,  7.50308275, 15.51740646]), array([0.]), array([ 0.       , 61.0365181]), array([  0.        ,  30.8740406 ,  58.74353027,  76.77220154,
       103.94380951, 143.86621094, 172.25131226, 215.28137207,
       249.97457886, 301.54013062, 348.16433716, 394.68099976,
       449.69744873, 486.60018921, 519.12640381, 546.65380859,
       577.72381592, 609.7348632

In [16]:
print(msd_input_dict)

{0: [], 1: [], 2: [], 3: []}


In [12]:
print(position_dict[atom_num]['location'])
print(position_dict[atom_num]['new_location'])

[3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 

In [ ]:
individual_msd = calculate_msd(position_dict[atom_num])
msd_dict[position_dict[atom_num]['location']][atom_num] = [individual_msd, len(individual_msd)]